In [ ]:
import pandas as pd
import duckdb
import os
from pathlib import Path
import holoviews as hv

from dask.distributed import Client, LocalCluster
from dask_gateway import Gateway
import dask.dataframe as dd
import teehr.queries.duckdb as tqd
import teehr.queries.pandas as tqp
import teehr.classes.teehr_duckdb as tds

In [ ]:
USGS_GEOMETRY = Path("/data/post-event/geo/usgs_point_geometry.conus.parquet")
CROSSWALK_FILEPATH = Path("/data/post-event/geo/usgs_nwm2*_crosswalk.conus.parquet")

# PRIMARY_FILEPATH = "s3://ciroh-rti-public-data/teehr/protocols/science-eval/timeseries/usgs/*.parquet"
# SECONDARY_FILEPATH = "s3://ciroh-rti-public-data/teehr/protocols/science-eval/timeseries/nwm2*/*.parquet"
# PRIMARY_FILEPATH = "s3://ciroh-rti-public-data/teehr/protocols/science-eval/timeseries/usgs_*.parquet"
# SECONDARY_FILEPATH = "s3://ciroh-rti-public-data/teehr/protocols/science-eval/timeseries/nwm2*.parquet"
# DATABASE_FILEPATH = "s3://ciroh-rti-public-data/teehr/protocols/science-eval/retro.db"

PRIMARY_FILEPATH = "/data/protocols/retro/timeseries/usgs_*.parquet"
SECONDARY_FILEPATH = "/data/protocols/retro/timeseries/nwm2*.parquet"
DATABASE_FILEPATH = "/data/protocols/retro/retro.db"

In [ ]:
# duckdb.query("""
#     INSTALL httpfs;
#     LOAD httpfs;
# """)

In [ ]:
# Need to configure storage_options

In [ ]:
# cluster = LocalCluster()
# client = Client(cluster)

In [ ]:
gateway = Gateway()

In [ ]:
options = gateway.cluster_options()
options.worker_cores = 1
options.worker_memory = 4
options  # should show interactive widget to select cores, etc

In [ ]:
cluster = gateway.new_cluster(options)

In [ ]:
client = cluster.get_client()
client

In [ ]:
cluster.scale(8)

In [ ]:
# pddf = dd.read_parquet(PRIMARY_FILEPATH, storage_options=storage_options, engine="pyarrow")
pddf = dd.read_parquet(PRIMARY_FILEPATH, engine="pyarrow")
pddf["location_id"] = pddf["location_id"].astype("category")
pddf["measurement_unit"] = pddf["measurement_unit"].astype("category")
pddf["configuration"] = pddf["measurement_unit"].astype("category")
pddf["variable_name"] = pddf["variable_name"].astype("category")
# pddf.set_index("value_time")
pddf

In [ ]:
sddf = dd.read_parquet(SECONDARY_FILEPATH, storage_options=storage_options)
sddf["location_id"] = pddf["location_id"].astype("category")
sddf["measurement_unit"] = pddf["measurement_unit"].astype("category")
sddf["configuration"] = pddf["measurement_unit"].astype("category")
sddf["variable_name"] = pddf["variable_name"].astype("category")
sddf.set_index("value_time")
sddf

In [ ]:
nwm20 = dd.read_parquet("/data/post-event/geo/usgs_nwm20_crosswalk.conus.parquet")
nwm21 = dd.read_parquet("/data/post-event/geo/usgs_nwm21_crosswalk.conus.parquet")
nwm22 = dd.read_parquet("/data/post-event/geo/usgs_nwm22_crosswalk.conus.parquet")
nwm20 = nwm20.reset_index().drop(columns=["feature_id"])
xw = dd.concat([nwm20, nwm21, nwm22])
xw["primary_location_id"] = xw["primary_location_id"].astype("category")
xw["secondary_location_id"] = xw["secondary_location_id"].astype("category")
xw

In [ ]:
sddf_xw = sddf.merge(xw, right_on="secondary_location_id", left_on="location_id")
sddf_xw

In [ ]:
joined = sddf_xw.merge(pddf, right_on=["location_id", "value_time"], left_on=["primary_location_id", "value_time"])
joined

In [ ]:
joined.groupby("primary_location_id", observed=False).value_x.sum().compute()

In [ ]:
pddf.groupby("location_id", observed=False).value.sum().compute()

In [ ]:
sddf.groupby("location_id", observed=False).value.sum().compute()

In [ ]:
cluster.shutdown()

In [ ]:
%%time
nwm = dd.read_parquet(PRIMARY_FILEPATH)
len(nwm)

In [ ]:
%%time
df = duckdb.query(f"""
SELECT * FROM read_parquet('{PRIMARY_FILEPATH}');
""").to_df()
df

In [ ]:
%%time
df = duckdb.query(f"""
SELECT * FROM read_parquet('{CROSSWALK_FILEPATH}');
""").to_df()
df

In [ ]:
%%time
df = duckdb.query(f"""
SELECT * FROM read_parquet('{SECONDARY_FILEPATH}');
""").to_df()
df

In [ ]:
%%time
df = tqd.get_metrics(
    primary_filepath=PRIMARY_FILEPATH,
    secondary_filepath=SECONDARY_FILEPATH,
    crosswalk_filepath=CROSSWALK_FILEPATH,
    include_metrics=["primary_count"],
    group_by=["primary_location_id", "configuration"],
    order_by=["primary_location_id", "configuration"],
    remove_duplicates=False,
    return_query=True,
)
print(df)

In [ ]:
%%time
df = tqp.get_metrics(
    primary_filepath=PRIMARY_FILEPATH,
    secondary_filepath=SECONDARY_FILEPATH,
    crosswalk_filepath=CROSSWALK_FILEPATH,
    include_metrics=["primary_count"],
    group_by=["primary_location_id", "configuration"],
    order_by=["primary_location_id", "configuration"],
    # filters = [
    #     {
    #         "column": "primary_location_id",
    #         "operator": "=",
    #         "value": "usgs-01010000",
    #     }
    # ]
)
df

In [ ]:
tdb = tds.TEEHRDatasetDB(DATABASE_FILEPATH)

In [ ]:
%%time
df = tdb.get_metrics(
    include_metrics=["primary_count"],
    group_by=["primary_location_id", "configuration"],
    order_by=["primary_location_id", "configuration"],
)
df

In [ ]:
%%time 
df = duckdb.query("""
 WITH joined AS (
            SELECT
                *
            FROM read_parquet('s3://ciroh-rti-public-data/teehr/protocols/science-eval/timeseries/joined_nwm2*.parquet')
        )
        , metrics AS (
            SELECT
                joined.primary_location_id,joined.configuration
                , count(primary_value) as primary_count
            FROM
                joined
            GROUP BY
                joined.primary_location_id,joined.configuration
        )
        SELECT
            metrics.*
        FROM metrics
        ORDER BY
            metrics.primary_location_id,metrics.configuration
    ;
    """).to_df()
df